In [ ]:
"""
‘Adapted from: “Integrating multimodal and multiscale connectivity blueprints of the human cerebral cortex in health and disease” [Hansen et al. PLoS Biology 2023] 

by Carlos Estevez-Fraga’

"""

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from netneurotools import datasets, stats
from scipy.spatial.distance import squareform, pdist
from scipy.stats import zscore, spearmanr
from scipy.optimize import curve_fit
from statsmodels.stats.multitest import multipletests


In [ ]:
def exponential(x, a, b, c):
    return a*np.exp(b*x)+c


def regress_dist(x, eu_distance, pars):
    return x - exponential(eu_distance, pars[0], pars[1], pars[2])


def corr_spin(x, y, spins, nspins):
    rho, _ = spearmanr(x, y, nan_policy='omit')
    null = np.zeros((nspins,))

    # null correlation
    for i in range(nspins):
        null[i], _ = spearmanr(x, y[spins[:, i]], nan_policy='omit')

    pval = (1 + sum(abs((null - np.mean(null))) >
                    abs((rho - np.mean(null))))) / (nspins + 1)
    return rho, pval


In [ ]:
"""
set-up
"""

path = '/Users/charlie/Desktop/my_projects/neurotransmitter/contributions/'

coords = np.genfromtxt(path+'data/parcellation_files/Cammoun033_coords.txt')
coords = coords[:, -3:]
nnodes = len(coords)

In [ ]:
# get cortical indices
info = pd.read_csv(datasets.fetch_cammoun2012()['info'])
hemiid = np.array(info.query('scale == "scale033" & structure == "cortex"')['hemisphere']) == 'R'
nnull = 1000
spins033 = stats.gen_spinsamples(coords, hemiid,
                                 n_rotate=nnull, seed=1234)
eu_distance = squareform(pdist(coords, metric="euclidean"))
mask = np.triu(np.ones(nnodes), 1) > 0

In [ ]:
# load enigma cortical abnormalities
# from https://github.com/netneurolab/hansen_crossdisorder_vulnerability
enigmapath = '/Users/charlie/Desktop/my_projects/neurotransmitter/ntrans/'
disease_profiles = zscore(np.genfromtxt(enigmapath+'data/enigma_atrophy/modified/modified_enigma_atrophy_ct.csv', delimiter=','))
disease_names = np.array(['ct_track', 'ct_trackon', 'ct_yas'])

In [ ]:
# load networks
gc = np.load(path+'data/Cammoun033/gene_coexpression.npy')
rs = np.load(path+'data/Cammoun033/receptor_similarity.npy')
ls = np.load(path+'data/Cammoun033/laminar_similarity.npy')
mc = np.load(path+'data/Cammoun033/metabolic_connectivity.npy')
hc = np.load(path+'data/Cammoun033/haemodynamic_connectivity.npy')
ec = np.load(path+'data/Cammoun033/electrophysiological_connectivity.npy')
ts = np.load(path+'data/Cammoun033/temporal_similarity.npy')

networks = {"gc" : gc,
            "rs" : rs,
            "ls" : ls,
            "mc" : mc,
            "hc" : hc,
            "ec" : ec,
            "ts" : ts}

# normalize networks
for network in networks.keys():
    networks[network] = np.arctanh(networks[network])
    networks[network][np.eye(nnodes).astype(bool)] = 0
    
# distance regressed version, for supplement
p0 = [1, -0.05, -0.1]  # initial parameter guess
pars = {}  # exponential curve variables
networks_reg = {}  # distance-regressed similarity matrices (upper triangular)
for network in networks.keys():
    pars[network], _ = curve_fit(exponential, eu_distance[mask],
                                 networks[network][mask], p0,
                                 bounds=([0, -10, -5], [10, 0, 5]))
    regressed = regress_dist(networks[network][mask], eu_distance[mask], pars[network])
    networks_reg[network] = np.zeros((nnodes, nnodes))
    networks_reg[network][np.triu(np.ones(nnodes, dtype=bool), 1)] = regressed
    networks_reg[network] = networks_reg[network] + networks_reg[network].T

# load SC
sc_wei = np.load(path+'data/Cammoun033/consensusSC_wei.npy')

# load colourmaps
cmap = np.genfromtxt(path+'data/colourmap.csv', delimiter=',')
cmap_div = ListedColormap(cmap)
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
"""
exposure analysis
"""

###TO change the colour##
# Define the number of colors you want to use
num_colors = 6

# Create a list of colors from a colormap
cmap = sns.color_palette("viridis", num_colors)

# Create a custom discrete color map
cmap_discrete = sns.color_palette(cmap, num_colors).as_hex()

nn_corrs = np.zeros((len(networks.keys()), len(disease_names), 2)) # rho, pval

# plot each correlation
fig, axs = plt.subplots(len(networks.keys()), len(disease_names),
                        sharex=True, figsize=(30, 13))
for d, disease in enumerate(disease_names):
    print(disease)
    for n, network in enumerate(networks.keys()):
        node_abnormality = disease_profiles[:, d]
        neighbour_abnormality = np.zeros((nnodes, ))
        net = networks[network].copy()
        net[net < 0] = 0
        for i in range(nnodes):  # for each node
            neighbour_abnormality[i] = np.sum(node_abnormality * net[i, :])/(sum(net[i, :] > 0))
        
        axs[n, d].scatter(node_abnormality, neighbour_abnormality, s=2)
        if n == len(networks.keys())-1:
            axs[n, d].set_xlabel(disease)
        if d == 0:
            axs[n, d].set_ylabel(network)
        axs[n, d].set_aspect(1.0/axs[n, d].get_data_ratio(), adjustable='box')
        # nn_corrs[n, d, 0] = spearmanr(node_abnormality, neighbour_abnormality)[0]
        nn_corrs[n, d, 0], nn_corrs[n, d, 1] = \
            corr_spin(node_abnormality, neighbour_abnormality, spins033, nnull)
        
# plt.tight_layout()
plt.savefig(path+'figures/Cammoun033/scatter_nncorr_NEG.eps')
np.save(path+'results/nn_corrs_networks_NEG.npy', nn_corrs)
nn_corrs[:, :, 1] = np.array([multipletests(nn_corrs[:, d, 1], method='fdr_bh')[1] \
                                for d in range(nn_corrs.shape[1])]).T

# plot heatmap
fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(nn_corrs[:, :, 0], annot=True, linewidths=.5,
            #cmap=cmap_div, vmin=-1, vmax=1,
            cmap=cmap_div, vmin=-1, vmax=1,
            xticklabels=disease_names, yticklabels=networks.keys())
plt.tight_layout()
plt.savefig(path+'figures/Cammoun033/heatmap_disease_nncorr_NEG.svg')


In [ ]:
# structure only
nn_corrs_sc = np.zeros((len(disease_names), 2))  # rho, pval
fig, axs = plt.subplots(1, len(disease_names), figsize=(30, 2))
for d, disease in enumerate(disease_names):
    print(disease)
    node_abnormality = disease_profiles[:, d]
    neighbour_abnormality = np.zeros((nnodes, ))
    for i in range(nnodes):
        neighbour_abnormality[i] = np.mean(node_abnormality[sc_wei[i, :] != 0]
                                           * sc_wei[sc_wei[i, :] != 0, i])
    axs[d].scatter(node_abnormality, neighbour_abnormality, s=2)
    axs[d].set_xlabel(disease)
    if d == 0:
        axs[d].set_ylabel('sc exposure')
    axs[d].set_aspect(1.0/axs[d].get_data_ratio(), adjustable='box')
    nn_corrs_sc[d, 0], nn_corrs_sc[d, 1] = \
        corr_spin(node_abnormality, neighbour_abnormality, spins033, nnull)
nn_corrs_sc[:, 1] = multipletests(nn_corrs_sc[:, 1], method='fdr_bh')[1]

fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(nn_corrs_sc[:, 0].reshape(1, -1), annot=True, linewidths=.5,
            cmap=cmap_div, vmin=-1, vmax=1, square=True,
            xticklabels=disease_names, yticklabels=['sc'])
plt.tight_layout()
plt.savefig(path+'figures/Cammoun033/heatmap_disease_nncorr_scd.svg')

In [ ]:
# distance only
nn_corrs_dist = np.zeros((len(disease_names), 2)) # rho, pval
eu_dist_rec = 1/eu_distance
eu_dist_rec[np.eye(nnodes, dtype=bool)] = 0
fig, axs = plt.subplots(1, len(disease_names), figsize=(30, 13))
for d, disease in enumerate(disease_names):
    print(disease)
    node_abnormality = disease_profiles[:, d]
    neighbour_abnormality = np.zeros((nnodes, ))
    for i in range(nnodes):  # for each node
        neighbour_abnormality[i] = np.sum(node_abnormality * eu_dist_rec[i, :])/(nnodes - 1)
    
    axs[d].scatter(node_abnormality, neighbour_abnormality, s=2)
    if d == 0:
        axs[d].set_ylabel('distance')
    axs[d].set_aspect(1.0/axs[d].get_data_ratio(), adjustable='box')
    nn_corrs_dist[d, 0], nn_corrs_dist[d, 1] = \
        corr_spin(node_abnormality, neighbour_abnormality, spins033, nnull)
nn_corrs_dist[:, 1] = multipletests(nn_corrs_dist[:, 1], method='fdr_bh')[1]

fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(nn_corrs_dist[:, 0].reshape(1, -1), annot=True, linewidths=.5,
            cmap=cmap_div, vmin=-1, vmax=1, square=True,
            xticklabels=disease_names, yticklabels=['dist'])
plt.tight_layout()
plt.savefig(path+'figures/Cammoun033/heatmap_disease_nncorr_dis_mind.svg')
